https://www.aemo.com.au/-/media/Files/Electricity/NEM/IT-Systems-and-Change/NEMDE-queue/NEMDE_Queue_Users_Guide.pdf

https://www.accc.gov.au/system/files/Appendix%206%20-%20HoustonKemp%20-%20Investigating%20wholesale%20market%20outcomes%20-%20Met....pdf

In [1]:
import xml.etree.ElementTree as ET
import os
import pandas as pd

def load_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()
    data = [c.attrib for c in root]
    df = pd.DataFrame(data)
    return df.query('Market == "Energy" and RegionID == "SA1" and DispatchedMarket == "ENOF"')

In [ ]:
from pathlib import Path
import nemdata

p = Path(nemdata.home).glob('**/NEMPriceSetter_*.xml')

fi = sorted([f for f in p])
print('found {}'.format(len(fi)))

data = []
for count, d in enumerate(fi):
    data.append(load_xml(d))
    
    if count % 5000 == 0:
        print('{:2.2f} pct'.format(100 * count / len(fi)))
        
data = pd.concat(data)

data.index = pd.to_datetime(data.loc[:, 'PeriodID'])
data.sort_index(inplace=True)

found 568226
0.00 pct
0.88 pct
1.76 pct


In [ ]:
data = data.sort_index()

data = data.astype({'Price': 'float32'})

data.head(10)

In [ ]:
data.loc[:, 'Price'].plot()

In [ ]:
gen = pd.read_csv('../generators_and_scheduled_loads.csv')

In [ ]:
f = pd.merge(data, gen, left_on='Unit', right_on='DUID')
f.set_index('PeriodID', drop=False, inplace=True)
f.columns

In [ ]:
carbon = pd.read_csv('../CO2EII_AVAILABLE_GENERATORS.CSV', skiprows=1)

final = pd.merge(f, carbon, left_on='DUID', right_on='DUID')
#final.set_index('PeriodID', drop=True, inplace=True)

In [ ]:
final.plot(y='CO2E_EMISSIONS_FACTOR')

In [ ]:
final.shape

In [ ]:
final.to_csv('final.csv', index=False)